This Notebook is a kaggle tutorial for Japanese kaggle beginners writen in Japanese.

# 3. ここで差がつく！ 仮説に基づいて新しい特徴量を作ってみよう

この[Notebook](https://www.kaggle.com/sishihara/upura-kaggle-tutorial-03-feature-engineering)では、特徴量エンジニアリングを学びます。

# 再現性の大切さ
「再現性がある」とは、何度実行しても同じ結果が得られることです。Kaggleで言うと、同一のスコアが得られると言い換えても良いでしょう。

再現性がないと、実行ごとに異なるスコアが得られてしまいます。今後、特徴量エンジニアリングなどでスコアの向上を試みても、予測モデルが改善されたか否かを正しく判断できなくなる問題が生じます。

実は、2つ目のNotebookには再現性がありませんでした。その原因は、Ageという特徴量の欠損値を埋める際の乱数です。ここでは標準偏差を考慮した乱数で欠損値を穴埋めしているのですが、この乱数は実行ごとに値が変わるようになってしまっています。

In [ ]:
# 前回のAgeを処理する部分までを実行

import numpy as np
import pandas as pd

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

`np.random.randint(age_avg - age_std, age_avg + age_std)` の実行ごとに、結果が異なります。

In [ ]:
age_avg = data['Age'].mean()
age_std = data['Age'].std()

np.random.randint(age_avg - age_std, age_avg + age_std)

In [ ]:
np.random.randint(age_avg - age_std, age_avg + age_std)

再現性を確保するためには、例えば次のような方法が考えられます。

1. そもそも乱数を用いる部分を削除する
2. 乱数のseedを与えて実行結果を固定する

Ageについては、そもそも乱数を用いるよりも、欠損していないデータの中央値を与えた方が筋の良い補完ができそうです。今回は中央値で補完するようにコードを改変します。

In [ ]:
data['Age'].fillna(data['Age'].median(), inplace=True)

In [ ]:
# その他の特徴量エンジニアリングの部分の処理

delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## 機械学習アルゴリズム

機械学習アルゴリズムの大半は乱数を利用するので、再現性を担保するためにはseedを設定しておかなければなりません。実は2つ目のKernelを振り返ると、機械学習アルゴリズムのロジスティック回帰のハイパーパラメータとして random_state=0 を与え、seedを固定していました。

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver="sag", random_state=0)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

このようにKaggleを進めていく際には、きちんと再現性が取れていることを随時確認していきましょう。（なお、GPUを利用する場合など、どうしても再現性が担保できない場合もあります）

# 仮説から新しい特徴量を作る

ここでは、実際に新しい特徴量を作っていきましょう。
例として探索的なデータ分析を実施した結果、ぼんやりと「一緒に乗船した家族の人数が多い方が、生存率が低そうだ」という仮説が得られた状況を考えます。

仮説が得られたので、次はこの仮説を検証するための可視化に移ります。新しい行「FamilySize」を作り、その大きさごとに生存したか否かを棒グラフにしましょう。

In [ ]:
# 改めてデータを読み込み直す
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)

In [ ]:
data.head()

「FamilySize」の作成に当たっては、ここまで削除していた「Parch」「SibSp」を使います。1を足しているのは、本人分です。

- Parch: 両親、子供の数
- SibSp: 兄弟、配偶者の数

In [ ]:
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
train['FamilySize'] = data['FamilySize'][:len(train)]
test['FamilySize'] = data['FamilySize'][len(train):]

import seaborn as sns
sns.countplot(x='FamilySize', data = train, hue='Survived')

ここでFamilySize >= 5の場合、死亡が生存を上回っており、生存率が低いことが分かります。

- Survived == 0: 死亡
- Survived == 1: 生存

「一緒に乗船した家族の人数が多い方が、生存率が低そうだ」という（ぼんやりとした）仮説が、可視化を通じて「FamilySize >= 5の場合、生存率が低いので、この特徴量は予測精度に寄与しそうだ」という確信を持った仮説に変わりました。

更に今回、可視化を通じて、それまで持っていなかった仮説（情報）を得ることもできました。「FamilySize == 1」の人が圧倒的に多く、かつ生存率が低いということです。

この「FamilySize == 1」であるという特徴量も予測精度に寄与しそうなので、下記のように新しく「IsAlone」という特徴量を作成してみましょう。

In [ ]:
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

train['IsAlone'] = data['IsAlone'][:len(train)]
test['IsAlone'] = data['IsAlone'][len(train):]

In [ ]:
# その他の特徴量エンジニアリングの部分の処理
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [ ]:
X_train.head()

## 予測精度の比較

新しい特徴量を加えた場合の予測精度を確認してみましょう。

In [ ]:
sub = gender_submission

In [ ]:
clf.fit(X_train, y_train)
y_pred_familysize_isalone = clf.predict(X_test)

sub['Survived'] = list(map(int, y_pred_familysize_isalone))
sub.to_csv("submission_familysize_isalone.csv", index = False)

sub.head()

比較のために、特徴量を加えなかった場合をいくつかのパターンで検証しておきます。

In [ ]:
clf.fit(X_train.drop('FamilySize', axis=1), y_train)
y_pred_isalone = clf.predict(X_test.drop('FamilySize', axis=1))

sub['Survived'] = list(map(int, y_pred_isalone))
sub.to_csv("submission_isalone.csv", index = False)

sub.head()

In [ ]:
clf.fit(X_train.drop('IsAlone', axis=1), y_train)
y_pred_familysize = clf.predict(X_test.drop('IsAlone', axis=1))

sub['Survived'] = list(map(int, y_pred_familysize))
sub.to_csv("submission_familysize.csv", index = False)

sub.head()

In [ ]:
clf.fit(X_train.drop(['FamilySize', 'IsAlone'], axis=1), y_train)
y_pred = clf.predict(X_test.drop(['FamilySize', 'IsAlone'], axis=1))

sub['Survived'] = list(map(int, y_pred))
sub.to_csv("submission.csv", index = False)

sub.head()